In [1]:

import pandas as pd
# ----------------------------- Load data -----------------------------
sessional_file = "Sessional_Data.xlsx"
grade_file = "Final_Grades_Data.xlsx"
df1 = pd.read_excel(sessional_file)
df2 = pd.read_excel(grade_file)


cols_to_delete = ['Name', 'Quiz 01', 'Quiz 02', 'Assig. 01', 'Assig. 02', 'Total']
df1.drop(columns = cols_to_delete, inplace=True)

cols_to_delete = ['Name of Student', 'Sessional (75)', 'Terminal (75)', 'Total 150']
df2.drop(columns = cols_to_delete, inplace=True)


df = pd.merge(df1, df2, on='Roll No.')

df.drop(columns=['Roll No.'], inplace=True)

# ----------------------------- Missing values -----------------------------

df = df.dropna()


# ----------------------------- Data Type Conversion -----------------------------

df['Mid Term'] = df['Mid Term'].astype(float)
df['Letter Grade'] = df['Letter Grade'].astype('category')


# print(df)

# ---------------------------- Outliers in Quiz AVG -----------------------------

q1 = df['Quiz AVG'].quantile(0.25)
q3 = df['Quiz AVG'].quantile(0.75)
IQR = q3 - q1
outliers = df[(df['Quiz AVG'] < q1 - 1.5 * IQR) | (df['Quiz AVG'] > q3 + 1.5 * IQR)]

if outliers.empty:
    pass
else:
    df = df.drop(outliers.index)

    
# -------------------------- Outliers in Assignment AVG -----------------------------

q1 = df['Assig. AVG'].quantile(0.25)
q3 = df['Assig. AVG'].quantile(0.75)
IQR = q3 - q1
outliers = df[(df['Assig. AVG'] < q1 - 1.5 * IQR) | (df['Assig. AVG'] > q3 + 1.5 * IQR)]

if outliers.empty:
    pass
else:
    df = df.drop(outliers.index)



# -------------------------- Outliers in Mid Term -----------------------------

q1 = df['Mid Term'].quantile(0.25)
q3 = df['Mid Term'].quantile(0.75)
IQR = q3 - q1
outliers = df[(df['Mid Term'] < q1 - 1.5 * IQR) | (df['Mid Term'] > q3 + 1.5 * IQR)]

if outliers.empty:
    pass
else:
    df = df.drop(outliers.index)
    
    
# ----------------------------- Dimensionality -----------------------------

# print("shape: ", df.shape)

rows = df.shape[0]
cols = df.shape[1]

# print("Number of samples (rows) are", rows)
# print("Number of features (columns) are", cols)

# ----------------------------- Descriptive Statistics -----------------------------
df.describe()




,Quiz AVG,Assig. AVG,Mid Term
count,204.000000,204.000000,204.000000
mean,7.933824,7.769608,28.995098
std,3.576302,3.631195,10.241901
min,1.500000,0.000000,6.000000
25%,5.000000,4.875000,22.000000
50%,7.500000,7.500000,30.000000
75%,10.125000,10.000000,37.000000
max,15.000000,15.000000,45.000000


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report

X = df[['Quiz AVG', 'Assig. AVG', 'Mid Term']]
y = df['Letter Grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

learning_rate = 0.001 
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 80, 40, 20), max_iter=1000, learning_rate_init=learning_rate, random_state=42)
mlp_classifier.fit(X_train, y_train)

y_predict = mlp_classifier.predict(X_test)

confusionMatrix = confusion_matrix(y_test, y_predict)
print('Confusion Matrix:')
print(confusionMatrix)

accuracy = accuracy_score(y_test, y_predict)
print('Accuracy:', accuracy)

precision = precision_score(y_test, y_predict, average='weighted')
print('Precision:', precision)

recall = recall_score(y_test, y_predict, average='weighted')
print('Recall:', recall)

f1 = f1_score(y_test, y_predict, average='weighted')
print('F1 Score:', f1)

report = classification_report(y_test, y_predict)
print('\nClassification Report:\n', report)


Confusion Matrix:
[[ 3  0  1  0  0]
 [ 0  1  1  0  0]
 [ 1  0  6  0  0]
 [ 0  0  0  8  1]
 [ 0  0  0  3 16]]
Accuracy: 0.8292682926829268
Precision: 0.8458001826007565
Recall: 0.8292682926829268
F1 Score: 0.8298102981029811

Classification Report:
               precision    recall  f1-score   support

           A       0.75      0.75      0.75         4
           B       1.00      0.50      0.67         2
          B+       0.75      0.86      0.80         7
           C       0.73      0.89      0.80         9
           F       0.94      0.84      0.89        19

    accuracy                           0.83        41
   macro avg       0.83      0.77      0.78        41
weighted avg       0.85      0.83      0.83        41



In [18]:
import joblib
joblib.dump(mlp_classifier, 'Final.joblib')

['Final.joblib']